# Preprocesamiento de datos

Preprocesado centrado en la resolución de esta serie de preguntas:

• ¿Cuánta gente a solicitado asilo en España a lo largo de los años y cuántas de esas solicitudes han sido aceptadas?

• ¿Y en el resto de países?

• ¿Cuál es el top 10 de países con mayor tasa de peticiones de asilo desde 2010? ¿Y cuál es el top 10 con menos?

• Dependiendo de los conflictos bélicos, por ejemplo, el conflicto Ucraniano y Ruso en 2022, ¿habrá aumentado el número de peticiones de estos dos países en comparación con los años anteriores?

• Mapas interactivos sobre peticiones de auxilio en España a lo largo de los años en cada una de las comunidades autónomas.

• En España, realizar comparaciones con una perspectiva de género en peticiones de auxilio.

In [3223]:
import pandas as pd
import numpy as np
from unidecode import unidecode

## Empezamos procesando las peticiones de asilo por países en España

Algunos métodos que se utilizaran para leer ficheros.

In [3224]:
def leer_fichero(filename, carpeta, ext, sheet=0, skip=-1, cols='A:D'):
    if ext == '.csv':
        asilo =  pd.read_csv(f'../data/raw/{carpeta}/{filename}', sep = ";", encoding='latin-1')
    else:
        if skip != -1:
            asilo = pd.read_excel(f'../data/raw/{carpeta}/{filename}', sheet_name=sheet, skiprows=skip, usecols=cols)
        else:
            asilo = pd.read_excel(f'../data/raw/{carpeta}/{filename}', sheet_name=sheet)
    return asilo

In [3225]:
def comprobar_pais(df, lista_paises):
    return df[~df['Nacionalidad '].isin(lista_paises)]

In [3226]:
def comprobar_pais_año(df, df_año):
    valor_faltante = df_año.loc[~df_año['Nacionalidad '].isin(df['Nacionalidad ']), 'Nacionalidad ']
    print(valor_faltante)

In [3227]:
def procesar_nacionalidad(df):
    df['Nacionalidad '] = df['Nacionalidad '].apply(unidecode)
    df['Nacionalidad '] = df['Nacionalidad '].str.lower()
    df['Nacionalidad '] = df['Nacionalidad '].str.strip()
    return df

In [3228]:
# Dataframe global
spain_df = pd.DataFrame()

### 2012 - España

In [3229]:
spain12 = leer_fichero('SilicitantesProteccionInternacional2012PorPaisescsv.csv', 'Asilo2012', '.csv')
spain12

,Nacionalidad,Hombres,Mujeres,Total
0,Afganistán,33.000,13.0,46.000
1,Albania,3.000,3.0,6.000
2,Angola,9.000,8.0,17.000
3,Argelia,174.000,28.0,202.000
4,Argentina,4.000,1.0,5.000
...,...,...,...,...
93,Venezuela,16.000,12.0,28.000
94,Yemen,3.000,2.0,5.000
95,Zambia,1.000,NaN,1.000
96,Zimbabwe,3.000,1.0,4.000


In [3230]:
spain12[spain12.isnull().any(axis=1)]

,Nacionalidad,Hombres,Mujeres,Total
6,Azerbaidyán,1.0,NaN,1.0
11,Bosnia Herzegovina,1.0,NaN,1.0
14,Burundi,1.0,NaN,1.0
20,Comoras,NaN,2.0,2.0
23,Costa Rica,1.0,NaN,1.0
24,Croacia,1.0,NaN,1.0
26,Ecuador,2.0,NaN,2.0
30,Estados Unidos de América,2.0,NaN,2.0
32,Francia,1.0,NaN,1.0
33,Gabón,NaN,1.0,1.0


Cambio a 0 los valores NaN y conversión del tipo de variable

In [3231]:
spain12 = spain12.fillna(0)
spain12.isnull().sum()

Nacionalidad     0
Hombres          0
Mujeres          0
Total            0
dtype: int64

Ponemos en minúscula y sin tildes la nacionalidad para ahorrar errores, en el df final se trabajará con ella

In [3232]:
spain12['Nacionalidad '] = spain12['Nacionalidad '].apply(unidecode)
spain12['Nacionalidad '] = spain12['Nacionalidad '].str.lower()
spain12.head()

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,33.0,13.0,46.0
1,albania,3.0,3.0,6.0
2,angola,9.0,8.0,17.0
3,argelia,174.0,28.0,202.0
4,argentina,4.0,1.0,5.0


#### Introducir solicitudes admitidas

In [3233]:
spain12_ad = leer_fichero('AdmisibilidaddeSolicitudesProtección InternacionalPorpaises.csv', 'Asilo2012', '.csv')
spain12_ad.head()

,Nacionalidad,Admitidas,No admitidas,Denegadas,Unnamed: 4
0,AFGANISTAN,45.0,1,0,NaN
1,ALBANIA,5.0,1,0,NaN
2,ANGOLA,6.0,1,10,NaN
3,ARGELIA,171.0,4,25,NaN
4,ARGENTINA,4.0,0,1,NaN


In [3234]:
spain12_ad['Nacionalidad '] = spain12_ad['Nacionalidad '].apply(unidecode)
spain12_ad['Nacionalidad '] = spain12_ad['Nacionalidad '].str.lower()
spain12_ad.head()

,Nacionalidad,Admitidas,No admitidas,Denegadas,Unnamed: 4
0,afganistan,45.0,1,0,NaN
1,albania,5.0,1,0,NaN
2,angola,6.0,1,10,NaN
3,argelia,171.0,4,25,NaN
4,argentina,4.0,0,1,NaN


In [3235]:
spain12_ad = spain12_ad.drop(columns=['No admitidas', 'Denegadas','Unnamed: 4'])

Hay varios paises en el que no se llaman igual en los archivos

In [3236]:
valor_faltante = spain12_ad.loc[~spain12_ad['Nacionalidad '].isin(spain12['Nacionalidad ']), 'Nacionalidad ']
valor_faltante

65     no reco. (palestina)
66       no recon. (sahara)
72                r.d.congo
73    republica dominicana 
85          trinidad tobago
Name: Nacionalidad , dtype: object

In [3237]:
# Quitar espacios finales
spain12_ad['Nacionalidad '] = spain12_ad['Nacionalidad '].str.strip()
spain12['Nacionalidad '] = spain12['Nacionalidad '].str.strip()

spain12_ad['Nacionalidad '] = spain12_ad['Nacionalidad '].replace('no reco. (palestina)', 'palestina')
spain12_ad['Nacionalidad '] = spain12_ad['Nacionalidad '].replace('no recon. (sahara)', 'sahara')
spain12_ad['Nacionalidad '] = spain12_ad['Nacionalidad '].replace('r.d.congo', 'r.d. congo')
spain12_ad['Nacionalidad '] = spain12_ad['Nacionalidad '].replace('trinidad tobago', 'trinidad y tobago')

spain12['Nacionalidad '] = spain12['Nacionalidad '].replace('no reconocido (palestina)', 'palestina')
spain12['Nacionalidad '] = spain12['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')

In [3238]:
valor_faltante = spain12.loc[~spain12['Nacionalidad '].isin(spain12_ad['Nacionalidad ']), 'Nacionalidad ']
print(valor_faltante)

60    mongolia
Name: Nacionalidad , dtype: object


Introduzco mongolia en el dataset de admitidos en la posición que tiene en el conjunto inicial

In [3239]:
nueva_fila = {'Nacionalidad ': 'mongolia', 'Admitidas': -1}
spain12_ad = pd.concat([spain12_ad.iloc[:60], pd.DataFrame([nueva_fila])], ignore_index=True)

In [3240]:
spain12_df = pd.merge(spain12, spain12_ad, on='Nacionalidad ')
spain12_df['Año'] = 2012
spain12_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,33.0,13.0,46.0,45.0,2012
1,albania,3.0,3.0,6.0,5.0,2012


In [3241]:
spain12_df.head(3)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,33.0,13.0,46.0,45.0,2012
1,albania,3.0,3.0,6.0,5.0,2012
2,angola,9.0,8.0,17.0,6.0,2012


In [3242]:
spain12_df = spain12_df.fillna(0)
spain12_df.isnull().sum()

Nacionalidad     0
Hombres          0
Mujeres          0
Total            0
Admitidas        0
Año              0
dtype: int64

In [3243]:
spain12_df['Admitidas'] = spain12_df['Admitidas'].astype('int64')
spain12_df['Hombres'] = spain12_df['Hombres'].astype('int64')
spain12_df['Mujeres'] = spain12_df['Mujeres'].astype('int64')
spain12_df['Total'] = spain12_df['Total'].astype('int64')

In [3244]:
spain12_df.shape

(61, 6)

### 2013 - España

In [3245]:
spain13 = pd.read_excel(f'../data/raw/Asilo2013.xlsx', sheet_name=0, skiprows=6, usecols='B:E')
spain13 = spain13.head(102)
spain13 = spain13.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain13

,Nacionalidad,Hombres,Mujeres,Total
0,Afganistán,48,18,66
1,Albania,7,9,16
2,Angola,5,2,7
3,Argelia,333,21,354
4,Argentina,2,0,2
...,...,...,...,...
97,Uzbekistán,4,7,11
98,Venezuela,20,15,35
99,Yemen,3,0,3
100,Zambia,1,0,1


In [3246]:
spain13['Nacionalidad '] = spain13['Nacionalidad '].apply(unidecode)
spain13['Nacionalidad '] = spain13['Nacionalidad '].str.lower()
spain13.head()

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,48,18,66
1,albania,7,9,16
2,angola,5,2,7
3,argelia,333,21,354
4,argentina,2,0,2


In [3247]:
valor_faltante = spain12.loc[~spain12['Nacionalidad '].isin(spain13['Nacionalidad ']), 'Nacionalidad ']
print(valor_faltante)

14              burundi
17                chile
23           costa rica
32              francia
43                india
59             moldavia
60             mongolia
61           montenegro
66            palestina
67               sahara
73           r.d. congo
86    trinidad y tobago
88         turkmenistan
97                total
Name: Nacionalidad , dtype: object


In [3248]:
spain13['Nacionalidad '] = spain13['Nacionalidad '].str.strip()
spain13['Nacionalidad '] = spain13['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain13['Nacionalidad '] = spain13['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain13['Nacionalidad '] = spain13['Nacionalidad '].replace('no reconocido (palestina)', 'palestina')
spain13['Nacionalidad '] = spain13['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')

In [3249]:
valor_faltante = spain13.loc[~spain13['Nacionalidad '].isin(spain12['Nacionalidad ']), 'Nacionalidad ']
print(valor_faltante)

8             belgica
13           botswana
16         cabo verde
23    corea del norte
33          filipinas
42             guyana
54           lituania
59           mauricio
62            myanmar
67             kosovo
78            rumania
83           singapur
89      sudan del sur
90         tayikistan
96            uruguay
Name: Nacionalidad , dtype: object


Estos países son los en los que personas pidieron refugio en 2013 y ninguno en 2012

In [3250]:
spain13_ad = pd.read_excel(f'../data/raw/Asilo2013.xlsx', sheet_name=7, skiprows=6, usecols='B:E')
spain13_ad = spain13_ad.head(102)
spain13_ad = spain13_ad.rename(columns={ 'País de origen ': 'Nacionalidad ' })
spain13_ad

,Nacionalidad,Admitidas,No admitidas,Denegadas
0,AFGANISTAN,63,2,1
1,ALBANIA,15,0,1
2,ANGOLA,2,0,5
3,ARGELIA,307,3,38
4,ARGENTINA,2,0,0
...,...,...,...,...
96,UZBEKISTAN,11,0,0
97,VENEZUELA,30,0,4
98,YEMEN,1,0,1
99,ZAMBIA,1,0,0


In [3251]:
spain13_ad = spain13_ad.drop(columns=['No admitidas', 'Denegadas'])

In [3252]:
spain13_ad.isnull().sum()

Nacionalidad     0
Admitidas        0
dtype: int64

In [3253]:
spain13_ad['Nacionalidad '] = spain13_ad['Nacionalidad '].apply(unidecode)
spain13_ad['Nacionalidad '] = spain13_ad['Nacionalidad '].str.lower()
spain13_ad['Nacionalidad '] = spain13_ad['Nacionalidad '].str.strip()
spain13_ad.head()

,Nacionalidad,Admitidas
0,afganistan,63
1,albania,15
2,angola,2
3,argelia,307
4,argentina,2


In [3254]:
valor_faltante = spain13.loc[~spain13['Nacionalidad '].isin(spain13_ad['Nacionalidad ']), 'Nacionalidad ']
print(valor_faltante)

50          kenia
67         kosovo
68      palestina
69         sahara
75     r.d. congo
101      zimbabwe
Name: Nacionalidad , dtype: object


In [3255]:
spain13_ad['Nacionalidad '] = spain13_ad['Nacionalidad '].replace('no recon. (kosovo)', 'kosovo')
spain13_ad['Nacionalidad '] = spain13_ad['Nacionalidad '].replace('no recon. (sahara)', 'sahara')
spain13_ad['Nacionalidad '] = spain13_ad['Nacionalidad '].replace('no reco. (palestina)', 'palestina')
spain13_ad['Nacionalidad '] = spain13_ad['Nacionalidad '].replace('republica democratica congo', 'r.d. congo')

In [3256]:
comprobar_pais_año(spain13_ad, spain13)

50        kenia
101    zimbabwe
Name: Nacionalidad , dtype: object


In [3257]:
kenia = {'Nacionalidad ': 'kenia', 'Admitidas': -1}
zimbabwe = {'Nacionalidad ': 'zimbabwe', 'Admitidas': -1}
spain13_ad = pd.concat([spain13_ad, pd.DataFrame([kenia])], ignore_index=True)
spain13_ad = pd.concat([spain13_ad, pd.DataFrame([zimbabwe])], ignore_index=True)

In [3258]:
spain13_ad.iloc[51]

Nacionalidad     liberia
Admitidas              6
Name: 51, dtype: object

In [3259]:
spain13_df = pd.merge(spain13, spain13_ad, on='Nacionalidad ')
spain13_df['Año'] = 2013
spain13_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,48,18,66,63,2013
1,albania,7,9,16,15,2013


In [3260]:
spain13_df.shape

(102, 6)

#### Concatenación de dataframes

In [3261]:
frames = [spain12_df, spain13_df]
spain_df = pd.concat(frames)
spain_df.shape

(163, 6)

In [3262]:
countries = sorted(list(set(spain_df['Nacionalidad '].values.tolist())))
countries[0:10]

['afganistan',
 'albania',
 'angola',
 'argelia',
 'argentina',
 'armenia',
 'azerbaidyan',
 'bangladesh',
 'belgica',
 'benin']

### 2014 - España

In [3263]:
spain14 = pd.read_excel(f'../data/raw/Asilo2014.xlsx', sheet_name=0, skiprows=6, usecols='B:E')
spain14 = spain14.head(102)
spain14 = spain14.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain14

,Nacionalidad,Hombres,Mujeres,Total
0,SIRIA,1032,650,1682
1,UCRANIA,478,468,946
2,MALI,614,6,620
3,ARGELIA,286,23,309
4,NO RECONOCIDO (PALESTINA),140,69,209
...,...,...,...,...
88,TURQUIA,1,0,1
89,VIETNAM,1,0,1
90,YEMEN,2,0,2
91,ZIMBABWE,0,1,1


In [3264]:
spain14['Nacionalidad '] = spain14['Nacionalidad '].apply(unidecode)
spain14['Nacionalidad '] = spain14['Nacionalidad '].str.lower()
spain14['Nacionalidad '] = spain14['Nacionalidad '].str.strip()
spain14.head()

,Nacionalidad,Hombres,Mujeres,Total
0,siria,1032,650,1682
1,ucrania,478,468,946
2,mali,614,6,620
3,argelia,286,23,309
4,no reconocido (palestina),140,69,209


In [3265]:
comprobar_pais(spain14, countries)

,Nacionalidad,Hombres,Mujeres,Total
4,no reconocido (palestina),140,69,209
26,no reconocido (sahara),27,4,31
27,r.d.congo,16,14,30
55,kazajistan,6,1,7
57,no reconocido (kosovo),5,2,7
63,apatrida,3,1,4
65,no reconocido (tibet),2,2,4
83,israel,1,0,1
84,montenegro,1,0,1
89,vietnam,1,0,1


In [3266]:
spain14['Nacionalidad '] = spain14['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain14['Nacionalidad '] = spain14['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain14['Nacionalidad '] = spain14['Nacionalidad '].replace('no reconocido (palestina)', 'palestina')
spain14['Nacionalidad '] = spain14['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain14['Nacionalidad '] = spain14['Nacionalidad '].replace('r.d.congo', 'r.d. congo')

In [3267]:
comprobar_pais(spain14, countries)

,Nacionalidad,Hombres,Mujeres,Total
55,kazajistan,6,1,7
63,apatrida,3,1,4
65,tibet,2,2,4
83,israel,1,0,1
84,montenegro,1,0,1
89,vietnam,1,0,1
92,total,4027,1923,5950


In [3268]:
spain14.isnull().sum()

Nacionalidad     0
Hombres          0
Mujeres          0
Total            0
dtype: int64

In [3269]:
spain14_ad = pd.read_excel(f'../data/raw/Asilo2014.xlsx', sheet_name=8, skiprows=6, usecols='B:E')
spain14_ad = spain14_ad.head(102)
spain14_ad = spain14_ad.rename(columns={ 'País de origen ': 'Nacionalidad ' })
spain14_ad

,Nacionalidad,Admitidas,No admitidas,Denegadas
0,AFGANISTAN,91,6,1
1,ALBANIA,7,0,4
2,ANGOLA,1,0,0
3,ARGELIA,212,3,88
4,ARGENTINA,1,0,1
...,...,...,...,...
87,VENEZUELA,108,0,5
88,VIETNAM,1,0,0
89,YEMEN,1,0,1
90,ZIMBABWE,1,0,0


In [3270]:
spain14_ad = procesar_nacionalidad(spain14_ad)
spain14_ad.head(3)

,Nacionalidad,Admitidas,No admitidas,Denegadas
0,afganistan,91,6,1
1,albania,7,0,4
2,angola,1,0,0


In [3271]:
spain14_ad = spain14_ad.drop(columns=['No admitidas', 'Denegadas'])

In [3272]:
comprobar_pais_año(spain14, spain14_ad)

60         no reconocido (kosovo)
61      no reconocido (palestina)
62         no reconocido (sahara)
63          no reconocido (tibet)
68    republica democratica congo
Name: Nacionalidad , dtype: object


In [3273]:
spain14_ad['Nacionalidad '] = spain14_ad['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain14_ad['Nacionalidad '] = spain14_ad['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain14_ad['Nacionalidad '] = spain14_ad['Nacionalidad '].replace('no reconocido (palestina)', 'palestina')
spain14_ad['Nacionalidad '] = spain14_ad['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain14_ad['Nacionalidad '] = spain14_ad['Nacionalidad '].replace('republica democratica congo', 'r.d. congo')

In [3274]:
comprobar_pais_año(spain14_ad, spain14)

63    apatrida
Name: Nacionalidad , dtype: object


In [3275]:
apatrida = {'Nacionalidad ': 'apatrida', 'Admitidas': -1}
spain14_ad = pd.concat([spain14_ad, pd.DataFrame([apatrida])], ignore_index=True)

In [3276]:
spain14_ad.isnull().sum()

Nacionalidad     0
Admitidas        0
dtype: int64

In [3277]:
spain14_df = pd.merge(spain14, spain14_ad, on='Nacionalidad ')
spain14_df['Año'] = 2014
spain14_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,siria,1032,650,1682,1567,2014
1,ucrania,478,468,946,813,2014


In [3278]:
spain14_df.shape

(93, 6)

In [3279]:
spain14_df.tail(1)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
92,total,4027,1923,5950,4897,2014


In [3280]:
spain14_df = spain14_df.iloc[:-1]

#### Añadir al df spain

In [3281]:
frames = [spain12_df, spain13_df, spain14_df]
spain_df = pd.concat(frames)
spain_df.shape

(255, 6)

In [3282]:
countries = sorted(list(set(spain_df['Nacionalidad '].values.tolist())))
countries[0:10]

['afganistan',
 'albania',
 'angola',
 'apatrida',
 'argelia',
 'argentina',
 'armenia',
 'azerbaidyan',
 'bangladesh',
 'belgica']

### 2015 - España

In [3283]:
spain15 = pd.read_excel(f'../data/raw/Asilo2015.xlsx', sheet_name=0, skiprows=6, usecols='B:E')
spain15 = spain15.head(102)
spain15 = spain15.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain15

,Nacionalidad,Hombres,Mujeres,Total
0,AFGANISTAN,31,10,41
1,ALBANIA,12,3,15
2,ANGOLA,7,2,9
3,ARGELIA,611,68,679
4,ARGENTINA,2,0,2
...,...,...,...,...
97,UGANDA,6,2,8
98,VENEZUELA,313,283,596
99,VIETNAM,3,0,3
100,YEMEN,9,5,14


In [3284]:
spain15 = procesar_nacionalidad(spain15)
spain15.head(3)

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,31,10,41
1,albania,12,3,15
2,angola,7,2,9


In [3285]:
spain15.isnull().sum()

Nacionalidad     0
Hombres          0
Mujeres          0
Total            0
dtype: int64

In [3286]:
comprobar_pais(spain15, countries)

,Nacionalidad,Hombres,Mujeres,Total
6,azerbaidjan,2,1,3
13,bulgaria,2,2,4
16,canada,1,0,1
17,centroafricana rep.,25,8,33
26,dominicana republica,87,192,279
49,kirguizia,0,1,1
50,laos,0,1,1
63,mozambique,0,2,2
69,no reconocido (kosovo),6,2,8
70,no reconocido (sahara),14,1,15


In [3287]:
spain15['Nacionalidad '] = spain15['Nacionalidad '].replace('azerbaidjan', 'azerbaidyan')
spain15['Nacionalidad '] = spain15['Nacionalidad '].replace('centroafricana rep.', 'republica centroafricana')
spain15['Nacionalidad '] = spain15['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain15['Nacionalidad '] = spain15['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain15['Nacionalidad '] = spain15['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain15['Nacionalidad '] = spain15['Nacionalidad '].replace('dominicana republica', 'republica dominicana')
spain15['Nacionalidad '] = spain15['Nacionalidad '].replace('r.d.congo', 'r.d. congo')
spain15['Nacionalidad '] = spain15['Nacionalidad '].replace('palestina eonu', 'palestina')

In [3288]:
comprobar_pais(spain15, countries)

,Nacionalidad,Hombres,Mujeres,Total
13,bulgaria,2,2,4
16,canada,1,0,1
49,kirguizia,0,1,1
50,laos,0,1,1
63,mozambique,0,2,2
72,oman,1,0,1
90,suriname,1,0,1
91,tanzania,1,0,1
101,total,9057,5830,14887


In [3289]:
spain15_ad = pd.read_excel(f'../data/raw/Asilo2015.xlsx', sheet_name=7, skiprows=6, usecols='B:E')
spain15_ad = spain15_ad.head(102)
spain15_ad = spain15_ad.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain15_ad

,Nacionalidad,Admitidas,No admitidas,Denegadas
0,AFGANISTAN,36,1,1
1,ALBANIA,10,0,4
2,ANGOLA,7,0,2
3,ARGELIA,454,7,205
4,ARGENTINA,0,0,2
...,...,...,...,...
97,UGANDA,7,0,1
98,VENEZUELA,567,0,6
99,VIETNAM,1,0,2
100,YEMEN,13,0,0


In [3290]:
spain15_ad = procesar_nacionalidad(spain15_ad)

In [3291]:
spain15_ad = spain15_ad.drop(columns=['No admitidas', 'Denegadas'])

In [3292]:
comprobar_pais_año(spain15, spain15_ad)

6                azerbaidjan
17       centroafricana rep.
26      dominicana republica
69    no reconocido (kosovo)
70    no reconocido (sahara)
71     no reconocido (tibet)
74            palestina eonu
78                 r.d.congo
Name: Nacionalidad , dtype: object


In [3293]:
spain15_ad['Nacionalidad '] = spain15_ad['Nacionalidad '].replace('azerbaidjan', 'azerbaidyan')
spain15_ad['Nacionalidad '] = spain15_ad['Nacionalidad '].replace('centroafricana rep.', 'republica centroafricana')
spain15_ad['Nacionalidad '] = spain15_ad['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain15_ad['Nacionalidad '] = spain15_ad['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain15_ad['Nacionalidad '] = spain15_ad['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain15_ad['Nacionalidad '] = spain15_ad['Nacionalidad '].replace('dominicana republica', 'republica dominicana')
spain15_ad['Nacionalidad '] = spain15_ad['Nacionalidad '].replace('r.d.congo', 'r.d. congo')
spain15_ad['Nacionalidad '] = spain15_ad['Nacionalidad '].replace('palestina eonu', 'palestina')

In [3294]:
comprobar_pais_año(spain15_ad, spain15)

Series([], Name: Nacionalidad , dtype: object)


In [3295]:
spain15_df = pd.merge(spain15, spain15_ad, on='Nacionalidad ')
spain15_df['Año'] = 2015
spain15_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,31,10,41,36,2015
1,albania,12,3,15,10,2015


In [3296]:
spain15_df = spain15_df.iloc[:-1]

#### Añadir al global

In [3297]:
frames = [spain12_df, spain13_df, spain14_df, spain15_df]
spain_df = pd.concat(frames)
spain_df.shape

(356, 6)

In [3298]:
countries = sorted(list(set(spain_df['Nacionalidad '].values.tolist())))
countries[0:10]

['afganistan',
 'albania',
 'angola',
 'apatrida',
 'argelia',
 'argentina',
 'armenia',
 'azerbaidyan',
 'bangladesh',
 'belgica']

### 2016 - España

In [3299]:
spain16 = leer_fichero('TABLA 2-1.xlsx', 'Asilo2016', '.xlsx', 0, 1, 'A:D')
spain16 = spain16.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain16

,Nacionalidad,Hombres,Mujeres,Total
0,Afganistán,49,20,69
1,Albania,24,2,26
2,Alemania,–,1,1
3,Angola,10,6,16
4,Arabia Saudí,1,–,1
...,...,...,...,...
99,Venezuela,2122,2074,4196
100,Vietnam,2,3,5
101,Yemen,38,10,48
102,Zimbabwe,–,2,2


In [3300]:
spain16.dtypes

Nacionalidad     object
Hombres          object
Mujeres          object
Total             int64
dtype: object

In [3301]:
spain16 = spain16.applymap(lambda x: x.rstrip() if isinstance(x, str) else x)
spain16 = spain16.replace("–", "0")

In [3302]:
spain16.head(5)

,Nacionalidad,Hombres,Mujeres,Total
0,Afganistán,49,20,69
1,Albania,24,2,26
2,Alemania,0,1,1
3,Angola,10,6,16
4,Arabia Saudí,1,0,1


In [3303]:
spain16 = procesar_nacionalidad(spain16)
spain16.head(3)

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,49,20,69
1,albania,24,2,26
2,alemania,0,1,1


In [3304]:
comprobar_pais(spain16, countries)

,Nacionalidad,Hombres,Mujeres,Total
2,alemania,0,1,1
4,arabia saudi,1,0,1
8,azerbaiyan,13,9,22
13,bosnia-herzegovina,1,0,1
26,desconocido,8,5,13
30,emiratos arabes unidos,1,0,1
38,grecia,1,0,1
41,guinea-bissau,8,1,9
52,kirguistan,1,0,1
68,no reconocido (kosovo),12,4,16


In [3305]:
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('bosnia-herzegovina', 'bosnia herzegovina')
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')
spain16['Nacionalidad '] = spain16['Nacionalidad '].replace('palestina eonu', 'palestina')

In [3306]:
comprobar_pais(spain16, countries)

,Nacionalidad,Hombres,Mujeres,Total
2,alemania,0,1,1
4,arabia saudi,1,0,1
26,desconocido,8,5,13
30,emiratos arabes unidos,1,0,1
38,grecia,1,0,1
103,total,9826,6718,16544


In [3307]:
spain16_ad = leer_fichero('TABLA 2-7.xlsx', 'Asilo2016', '.xlsx', 0, 2, 'A:D')
spain16_ad = spain16_ad.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain16_ad

,Nacionalidad,Admitidas,No admitidas,Denegadas
0,Afganistán,69,–,–
1,Albania,14,–,9
2,Alemania,–,1,–
3,Angola,6,–,10
4,Arabia Saudí,1,–,–
...,...,...,...,...
99,Venezuela,4077,–,38
100,Vietnam,5,–,–
101,Yemen,46,–,–
102,Zimbabwe,2,–,–


In [3308]:
spain16_ad = procesar_nacionalidad(spain16_ad)
spain16_ad.head(3)

,Nacionalidad,Admitidas,No admitidas,Denegadas
0,afganistan,69,–,–
1,albania,14,–,9
2,alemania,–,1,–


In [3309]:
spain16_ad = spain16_ad.applymap(lambda x: x.rstrip() if isinstance(x, str) else x)
spain16_ad = spain16_ad.applymap(lambda x: x.lstrip() if isinstance(x, str) else x)
spain16_ad = spain16_ad.replace("–", "0")
spain16_ad.head(2)

,Nacionalidad,Admitidas,No admitidas,Denegadas
0,afganistan,69,0,0
1,albania,14,0,9


In [3310]:
spain16_ad = spain16_ad.drop(columns=['No admitidas', 'Denegadas'])

In [3311]:
comprobar_pais_año(spain16, spain16_ad)

8                          azerbaiyan
13                 bosnia-herzegovina
41                      guinea-bissau
52                         kirguistan
68             no reconocido (kosovo)
69             no reconocido (sahara)
70              no reconocido (tibet)
73                     palestina eonu
78    republica democratica del congo
Name: Nacionalidad , dtype: object


In [3312]:
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('bosnia-herzegovina', 'bosnia herzegovina')
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')
spain16_ad['Nacionalidad '] = spain16_ad['Nacionalidad '].replace('palestina eonu', 'palestina')

In [3313]:
comprobar_pais_año(spain16_ad, spain16)

Series([], Name: Nacionalidad , dtype: object)


In [3314]:
spain16_df = pd.merge(spain16, spain16_ad, on='Nacionalidad ')
spain16_df['Año'] = 2016
spain16_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,49,20,69,69,2016
1,albania,24,2,26,14,2016


In [3315]:
spain16_df = spain16_df.iloc[:-1]

#### Añadimos al normal

In [3316]:
frames = [spain12_df, spain13_df, spain14_df, spain15_df, spain16_df]
spain_df = pd.concat(frames)
spain_df.shape

(459, 6)

In [3317]:
countries = sorted(list(set(spain_df['Nacionalidad '].values.tolist())))
countries[0:10]

['afganistan',
 'albania',
 'alemania',
 'angola',
 'apatrida',
 'arabia saudi',
 'argelia',
 'argentina',
 'armenia',
 'azerbaidyan']

### 2017 - España

In [3318]:
spain17 = leer_fichero('TABLA 2-1.xlsx', 'Asilo2017', '.xlsx', 0, 1, 'A:D')
#warnings.simplefilter(action='ignore', category=UserWarning)
#return pd.read_excel(path)
spain17 = spain17.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain17

C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Nacionalidad,Hombres,Mujeres,Total
0,Afganistán,100,24,124
1,Albania,42,14,56
2,Angola,9,11,20
3,Apátrida,1,2,3
4,Arabia Saudí,0,2,2
...,...,...,...,...
108,Venezuela,5151,5478,10629
109,Vietnam,32,14,46
110,Yemen,132,42,174
111,Zambia,0,1,1


In [3319]:
spain17 = procesar_nacionalidad(spain17)
spain17.head(3)

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,100,24,124
1,albania,42,14,56
2,angola,9,11,20


In [3320]:
comprobar_pais(spain17, countries)

,Nacionalidad,Hombres,Mujeres,Total
8,azerbaiyan,15,12,27
13,bosnia y herzegovina,2,2,4
24,comores,1,0,1
30,djibouti,0,1,1
44,guinea-bissau,19,4,23
49,indonesia,0,1,1
57,kirguistan,6,5,11
74,no reconocido (kosovo),2,0,2
75,no reconocido (sahara),254,4,258
76,no reconocido (tibet),3,1,4


In [3321]:
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('palestina eonu', 'palestina')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('comores', 'comoras')
spain17['Nacionalidad '] = spain17['Nacionalidad '].replace('palestina eonu', 'palestina')

In [3322]:
comprobar_pais(spain17, countries)

,Nacionalidad,Hombres,Mujeres,Total
30,djibouti,0,1,1
49,indonesia,0,1,1
82,reino unido,1,0,1
96,suecia,1,0,1
100,trinidad y tobago,4,0,4
102,turkmenistan,0,3,3
112,total,18271,13469,31740


In [3323]:
spain17_ad = leer_fichero('TABLA 2-8.xlsx', 'Asilo2017', '.xlsx', 0, 2, 'A:G')
spain17_ad

C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Unnamed: 0,Hombres,Mujeres,Hombres.1,Mujeres.1,Hombres.2,Mujeres.2
0,Afganistán,95,24,0,0,0,0
1,Albania,30,14,0,0,8,0
2,Angola,7,8,0,0,2,3
3,Apátrida,1,2,0,0,0,0
4,Arabia Saudí,0,1,0,0,0,1
...,...,...,...,...,...,...,...
108,Venezuela,5030,5329,1,0,15,14
109,Vietnam,20,5,0,0,12,9
110,Yemen,131,42,0,0,1,0
111,Zambia,0,1,0,0,0,0


In [3324]:
spain17_ad = spain17_ad.rename(columns={'Unnamed: 0': 'Nacionalidad ' })
spain17_ad = spain17_ad.drop(columns=['Hombres.1', 'Mujeres.1',	'Hombres.2', 'Mujeres.2'])
spain17_ad

,Nacionalidad,Hombres,Mujeres
0,Afganistán,95,24
1,Albania,30,14
2,Angola,7,8
3,Apátrida,1,2
4,Arabia Saudí,0,1
...,...,...,...
108,Venezuela,5030,5329
109,Vietnam,20,5
110,Yemen,131,42
111,Zambia,0,1


In [3325]:
spain17_ad['Admitidas'] = spain17_ad['Hombres'] + spain17_ad['Mujeres']
spain17_ad

,Nacionalidad,Hombres,Mujeres,Admitidas
0,Afganistán,95,24,119
1,Albania,30,14,44
2,Angola,7,8,15
3,Apátrida,1,2,3
4,Arabia Saudí,0,1,1
...,...,...,...,...
108,Venezuela,5030,5329,10359
109,Vietnam,20,5,25
110,Yemen,131,42,173
111,Zambia,0,1,1


In [3326]:
spain17_ad = spain17_ad.drop(columns=['Hombres', 'Mujeres'])
spain17_ad

,Nacionalidad,Admitidas
0,Afganistán,119
1,Albania,44
2,Angola,15
3,Apátrida,3
4,Arabia Saudí,1
...,...,...
108,Venezuela,10359
109,Vietnam,25
110,Yemen,173
111,Zambia,1


In [3327]:
spain17_ad = procesar_nacionalidad(spain17_ad)
spain17_ad.head(3)

,Nacionalidad,Admitidas
0,afganistan,119
1,albania,44
2,angola,15


In [3328]:
comprobar_pais_año(spain17, spain17_ad)

8                          azerbaiyan
13               bosnia y herzegovina
24                            comores
44                      guinea-bissau
57                         kirguistan
74             no reconocido (kosovo)
75             no reconocido (sahara)
76              no reconocido (tibet)
78                     palestina eonu
84    republica democratica del congo
Name: Nacionalidad , dtype: object


In [3329]:
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('palestina eonu', 'palestina')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('comores', 'comoras')
spain17_ad['Nacionalidad '] = spain17_ad['Nacionalidad '].replace('palestina eonu', 'palestina')

In [3330]:
comprobar_pais_año(spain17_ad, spain17)

Series([], Name: Nacionalidad , dtype: object)


In [3331]:
spain17_df = pd.merge(spain17, spain17_ad, on='Nacionalidad ')
spain17_df['Año'] = 2017
spain17_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,100,24,124,119,2017
1,albania,42,14,56,44,2017


In [3332]:
spain17_df = spain17_df.iloc[:-1]

#### Añadir al dataframe global

In [3333]:
frames = [spain12_df, spain13_df, spain14_df, spain15_df, spain16_df, spain17_df]
spain_df = pd.concat(frames)
spain_df.shape

(571, 6)

In [3334]:
countries = sorted(list(set(spain_df['Nacionalidad '].values.tolist())))
countries[0:10]

['afganistan',
 'albania',
 'alemania',
 'angola',
 'apatrida',
 'arabia saudi',
 'argelia',
 'argentina',
 'armenia',
 'azerbaidyan']

### 2018 - España

In [3335]:
spain18 = leer_fichero('TABLA 2-1.xlsx', 'Asilo2018', '.xlsx', 0, 1, 'A:D')
spain18 = spain18.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain18.shape

(123, 4)

In [3336]:
spain18 = procesar_nacionalidad(spain18)
spain18.head(3)

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,78,23,101
1,albania,42,21,63
2,angola,10,14,24


In [3337]:
comprobar_pais(spain18, countries)

,Nacionalidad,Hombres,Mujeres,Total
8,azerbaiyan,11,8,19
12,butan,0,1,1
15,bosnia y herzegovina,1,0,1
16,botsuana,1,0,1
26,comores,2,4,6
48,guinea-bissau,37,3,40
57,italia,2,1,3
62,kirguistan,0,3,3
68,malasia,0,1,1
80,no reconocido (kosovo),1,0,1


In [3338]:
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('comores', 'comoras')

spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('palestina eonu', 'palestina')

spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('palestina eonu', 'palestina')
spain18['Nacionalidad '] = spain18['Nacionalidad '].replace('botsuana', 'botswana')

In [3339]:
comprobar_pais(spain18, countries)

,Nacionalidad,Hombres,Mujeres,Total
12,butan,0,1,1
57,italia,2,1,3
68,malasia,0,1,1
81,no reconocido (kurdo),1,0,1
89,polonia,1,0,1
108,taiwan,192,27,219
109,tadyikistan,2,0,2
122,total,31964,23785,55749


In [3340]:
spain18_ad = leer_fichero('TABLA 2-6.xlsx', 'Asilo2018', '.xlsx', 0, 2, 'A:G')
spain18_ad.head(3)

,Unnamed: 0,H,M,H.1,M.1,H.2,M.2
0,Afganistán,71,22,0,0,0,0
1,Albania,30,19,0,0,11,0
2,Angola,9,12,0,0,1,2


In [3341]:
spain18_ad.shape

(122, 7)

In [3342]:
spain18_ad = spain18_ad.rename(columns={'Unnamed: 0': 'Nacionalidad ' })
spain18_ad = spain18_ad.drop(columns=['H.1', 'M.1',	'H.2', 'M.2'])
spain18_ad.head(3)

C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,Nacionalidad,H,M
0,Afganistán,71,22
1,Albania,30,19
2,Angola,9,12


In [3343]:
spain18_ad['Admitidas'] = spain18_ad['H'] + spain18_ad['M']
spain18_ad.head(3)

,Nacionalidad,H,M,Admitidas
0,Afganistán,71,22,93
1,Albania,30,19,49
2,Angola,9,12,21


In [3344]:
spain18_ad = spain18_ad.drop(columns=['H', 'M'])
spain18_ad.head(3)

,Nacionalidad,Admitidas
0,Afganistán,93
1,Albania,49
2,Angola,21


In [3345]:
spain18_ad = procesar_nacionalidad(spain18_ad)
spain18_ad.head(3)

,Nacionalidad,Admitidas
0,afganistan,93
1,albania,49
2,angola,21


In [3346]:
comprobar_pais_año(spain18, spain18_ad)

8                           azerbaiyan
14                bosnia y herzegovina
15                            botsuana
25                             comores
47                       guinea-bissau
61                          kirguistan
79              no reconocido (kosovo)
81              no reconocido (sahara)
82               no reconocido (tibet)
84                      palestina eonu
91     republica democratica del congo
106                           tanzania
109                  trinidad y tobago
Name: Nacionalidad , dtype: object


In [3347]:
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('palestina eonu', 'palestina')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('comores', 'comoras')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('palestina eonu', 'palestina')
spain18_ad['Nacionalidad '] = spain18_ad['Nacionalidad '].replace('botsuana', 'botswana')


In [3348]:
comprobar_pais_año(spain18, spain18_ad)

106             tanzania
109    trinidad y tobago
Name: Nacionalidad , dtype: object


In [3349]:
spain18_ad = spain18_ad.loc[spain18_ad['Nacionalidad '] != 'tanzania']
spain18_ad = spain18_ad.loc[spain18_ad['Nacionalidad '] != 'trinidad y tobago']

In [3350]:
comprobar_pais_año(spain18_ad, spain18)

12             butan
100         singapur
106    sudan del sur
Name: Nacionalidad , dtype: object


In [3351]:
butan = {'Nacionalidad ': 'butan', 'Admitidas': -1}
singapur = {'Nacionalidad ': 'singapur', 'Admitidas': -1}
sudan = {'Nacionalidad ': 'sudan del sur', 'Admitidas': -1}

spain18_ad = pd.concat([spain18_ad, pd.DataFrame([butan])], ignore_index=True)
spain18_ad = pd.concat([spain18_ad, pd.DataFrame([singapur])], ignore_index=True)
spain18_ad = pd.concat([spain18_ad, pd.DataFrame([sudan])], ignore_index=True)

In [3352]:
spain18_df = pd.merge(spain18, spain18_ad, on='Nacionalidad ')
spain18_df['Año'] = 2018
spain18_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,78,23,101,93,2018
1,albania,42,21,63,49,2018


In [3353]:
spain18_df = spain18_df.iloc[:-1]

#### Merge

In [3354]:
frames = [spain12_df, spain13_df, spain14_df, spain15_df, spain16_df, spain17_df, spain18_df]
spain_df = pd.concat(frames)
spain_df.shape

(693, 6)

### 2019 - España

In [3355]:
spain19 = pd.read_excel(f'../data/raw/Asilo2019.xlsx', sheet_name=1, skiprows=1, usecols='A:D')
spain19 = spain19.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain19

,Nacionalidad,Hombres,Mujeres,Total
0,Afganistán,120,62,182
1,Albania,130,32,162
2,Alemania,3,0,3
3,Angola,20,25,45
4,Apátridas,3,0,3
...,...,...,...,...
117,Vietnam,39,13,52
118,Yemen,425,61,486
119,Zambia,1,0,1
120,Zimbabwe,1,1,2


In [3356]:
spain19 = procesar_nacionalidad(spain19)
spain19.head(3)

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,120,62,182
1,albania,130,32,162
2,alemania,3,0,3


In [3357]:
comprobar_pais(spain19, countries)

,Nacionalidad,Hombres,Mujeres,Total
4,apatridas,3,0,3
9,azerbaiyan,23,12,35
10,bahrein,0,1,1
15,bosnia y herzegovina,4,0,4
32,dominica,0,1,1
48,guinea-bissau,33,4,37
57,italia,1,0,1
62,kirguistan,8,7,15
63,kuwait,1,2,3
80,no reconocido (kosovo),3,0,3


In [3358]:
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('apatridas', 'apatrida')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('dominica', 'republica dominicana')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('palestina eonu', 'palestina')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain19['Nacionalidad '] = spain19['Nacionalidad '].replace('kirguistan', 'kirguizia')

In [3359]:
comprobar_pais(spain19, countries)

,Nacionalidad,Hombres,Mujeres,Total
10,bahrein,0,1,1
57,italia,1,0,1
63,kuwait,1,2,3
88,portugal,2,1,3
91,republica democratica del congo,48,44,92
105,taiwan,1,0,1
121,total,64630,53816,118446


In [3360]:
spain19_ad = pd.read_excel(f'../data/raw/Asilo2019.xlsx', sheet_name=6, skiprows=2, usecols='A:G')
spain19_ad = spain19_ad.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain19_ad

,Unnamed: 0,H,M,H.1,M.1,H.2,M.2
0,Afganistán,9,12,34,9,14,5
1,Albania,0,0,0,0,26,8
2,Alemania,0,0,0,0,1,0
3,Angola,0,0,0,0,5,5
4,Apátridas,0,0,0,0,0,1
...,...,...,...,...,...,...,...
106,Venezuela,21,27,0,0,144,37
107,Vietnam,0,0,0,0,2,2
108,Yemen,1,0,96,6,132,38
109,Zambia,0,1,0,0,0,0


In [3361]:
spain19_ad = spain19_ad.rename(columns={'Unnamed: 0': 'Nacionalidad ' })
spain19_ad = spain19_ad.drop(columns=['H.1', 'M.1',	'H.2', 'M.2'])
spain19_ad.head(2)

,Nacionalidad,H,M
0,Afganistán,9,12
1,Albania,0,0


In [3362]:
spain19_ad['Admitidas'] = spain19_ad['H'] + spain19_ad['M']
spain19_ad = spain19_ad.drop(columns=['H', 'M'])
spain19_ad.head(2)

,Nacionalidad,Admitidas
0,Afganistán,21
1,Albania,0


In [3363]:
spain19_ad = procesar_nacionalidad(spain19_ad)
spain19_ad.head(3)

,Nacionalidad,Admitidas
0,afganistan,21
1,albania,0
2,alemania,0


In [3364]:
comprobar_pais_año(spain19, spain19_ad)

4                   apatridas
9                  azerbaiyan
14       bosnia y herzegovina
39                     grecia
43              guinea-bissau
54                 kirguistan
58                   lituania
59                  macedonia
60                    malasia
68                    myanmar
73     no reconocido (kosovo)
74     no reconocido (sahara)
75      no reconocido (tibet)
77             palestina eonu
94              sudan del sur
100              turkmenistan
Name: Nacionalidad , dtype: object


In [3365]:
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('apatridas', 'apatrida')
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('palestina eonu', 'palestina')
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain19_ad['Nacionalidad '] = spain19_ad['Nacionalidad '].replace('kirguistan', 'kirguizia')

In [3366]:
comprobar_pais_año(spain19, spain19_ad)

39            grecia
58          lituania
59         macedonia
60           malasia
68           myanmar
75             tibet
94     sudan del sur
100     turkmenistan
Name: Nacionalidad , dtype: object


In [3367]:
spain19_ad = spain19_ad.loc[spain19_ad['Nacionalidad '] != 'grecia']
spain19_ad = spain19_ad.loc[spain19_ad['Nacionalidad '] != 'tibet']
spain19_ad = spain19_ad.loc[spain19_ad['Nacionalidad '] != 'macedonia']
spain19_ad = spain19_ad.loc[spain19_ad['Nacionalidad '] != 'lituania']
spain19_ad = spain19_ad.loc[spain19_ad['Nacionalidad '] != 'malasia']
spain19_ad = spain19_ad.loc[spain19_ad['Nacionalidad '] != 'myanmar']
spain19_ad = spain19_ad.loc[spain19_ad['Nacionalidad '] != 'sudan del sur']
spain19_ad = spain19_ad.loc[spain19_ad['Nacionalidad '] != 'turkmenistan']

In [3368]:
comprobar_pais_año(spain19_ad, spain19_ad)

Series([], Name: Nacionalidad , dtype: object)


In [3369]:
spain19_df = pd.merge(spain19, spain19_ad, on='Nacionalidad ')
spain19_df['Año'] = 2019
spain19_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,120,62,182,21,2019
1,albania,130,32,162,0,2019


In [3370]:
spain19_df = spain19_df.iloc[:-1]

In [3371]:
frames = [spain12_df, spain13_df, spain14_df, spain15_df, spain16_df, spain17_df, spain18_df, spain19_df]
spain_df = pd.concat(frames)
spain_df.shape

(796, 6)

In [3372]:
countries = sorted(list(set(spain_df['Nacionalidad '].values.tolist())))
countries[0:10]

['afganistan',
 'albania',
 'alemania',
 'angola',
 'apatrida',
 'arabia saudi',
 'argelia',
 'argentina',
 'armenia',
 'azerbaidyan']

### 2020 - España

In [3373]:
spain20 = pd.read_excel(f'../data/raw/Asilo2020.xlsx', sheet_name=1, skiprows=1, usecols='A:D')
spain20 = spain20.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain20

,Nacionalidad,Hombres,Mujeres,Total
0,Afganistán,27.0,7.0,34.0
1,Albania,37.0,13.0,50.0
2,Alemania,1.0,0.0,1.0
3,Angola,13.0,11.0,24.0
4,Apátrida,5.0,2.0,7.0
...,...,...,...,...
109,Vietnam,14.0,9.0,23.0
110,Yemen,117.0,29.0,146.0
111,Total,47094.0,41732.0,88826.0
112,NaN,NaN,NaN,NaN


In [3374]:
spain20 = spain20.fillna(0)
spain20.isnull().sum()

Nacionalidad     0
Hombres          0
Mujeres          0
Total            0
dtype: int64

In [3375]:
spain20 = spain20.iloc[:-2]
spain20['Nacionalidad '].unique()

array(['Afganistán', 'Albania', 'Alemania', 'Angola', 'Apátrida',
       'Arabia Saudí', 'Argelia', 'Argentina', 'Armenia', 'Azerbaiyán',
       'Bahréin', 'Bangladesh', 'Belice', 'Benín', 'Bielorrusia',
       'Bolivia', 'Bosnia y Herzegovina', 'Brasil', 'Burkina Faso',
       'Burundi', 'Camerún', 'Canadá', 'Chad', 'Chile', 'China',
       'Colombia', 'Comores', 'Congo', 'Corea', 'Costa de Marfil',
       'Costa Rica', 'Cuba', 'Desconocido', 'Ecuador', 'Egipto',
       'El Salvador', 'Emiratos Árabes Unidos', 'Eritrea',
       'Estados Unidos de América', 'Etiopía', 'Filipinas', 'Francia',
       'Gabón', 'Gambia', 'Georgia', 'Ghana', 'Guatemala', 'Guinea',
       'Guinea-Bissau', 'Guinea Ecuatorial', 'Haití', 'Honduras', 'India',
       'Irán', 'Iraq', 'Israel', 'Jamaica', 'Japón', 'Jordania',
       'Kazajstán', 'Kenia', 'Kirguistán', 'Kuwait', 'Líbano', 'Liberia',
       'Libia', 'Macedonia', 'Mali', 'Marruecos', 'Mauritania', 'México',
       'Moldavia', 'Mongolia', 'Mozambique',

In [3376]:
spain20 = procesar_nacionalidad(spain20)
spain20.head(3)

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,27.0,7.0,34.0
1,albania,37.0,13.0,50.0
2,alemania,1.0,0.0,1.0


In [3377]:
comprobar_pais(spain20, countries)

,Nacionalidad,Hombres,Mujeres,Total
9,azerbaiyan,9.0,6.0,15.0
10,bahrein,1.0,0.0,1.0
12,belice,1.0,1.0,2.0
16,bosnia y herzegovina,0.0,1.0,1.0
26,comores,1.0,2.0,3.0
28,corea,2.0,0.0,2.0
48,guinea-bissau,20.0,3.0,23.0
57,japon,0.0,1.0,1.0
61,kirguistan,1.0,0.0,1.0
62,kuwait,2.0,0.0,2.0


In [3378]:
spain20['Nacionalidad '] = spain20['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain20['Nacionalidad '] = spain20['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain20['Nacionalidad '] = spain20['Nacionalidad '].replace('comores', 'comoras')
spain20['Nacionalidad '] = spain20['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain20['Nacionalidad '] = spain20['Nacionalidad '].replace('no reconocido (desconocido)', 'desconocido')
spain20['Nacionalidad '] = spain20['Nacionalidad '].replace('palestina eonu', 'palestina')
spain20['Nacionalidad '] = spain20['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain20['Nacionalidad '] = spain20['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')

comprobar_pais(spain20, countries)

,Nacionalidad,Hombres,Mujeres,Total
10,bahrein,1.0,0.0,1.0
12,belice,1.0,1.0,2.0
28,corea,2.0,0.0,2.0
57,japon,0.0,1.0,1.0
62,kuwait,2.0,0.0,2.0
98,tailandia,0.0,1.0,1.0
111,total,47094.0,41732.0,88826.0


In [3379]:
spain20_ad = pd.read_excel(f'../data/raw/Asilo2020.xlsx', sheet_name=6, skiprows=2, usecols='A:G')
spain20_ad = spain20_ad.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain20_ad = spain20_ad.iloc[:-2]
spain20_ad.tail(5)

,Unnamed: 0,H,M,H.1,M.1,H.2,M.2
100,Venezuela,0.0,5.0,0.0,0.0,566.0,344.0
101,Vietnam,0.0,0.0,0.0,0.0,11.0,10.0
102,Yemen,13.0,9.0,83.0,5.0,230.0,36.0
103,Zimbabwe,0.0,0.0,0.0,0.0,0.0,1.0
104,Total,2030.0,2329.0,901.0,500.0,41691.0,30528.0


In [3380]:
spain20_ad = spain20_ad.rename(columns={'Unnamed: 0': 'Nacionalidad ' })
spain20_ad = spain20_ad.drop(columns=['H.1', 'M.1',	'H.2', 'M.2'])
spain20_ad.head(2)

,Nacionalidad,H,M
0,Afganistán,4.0,5.0
1,Albania,0.0,0.0


In [3381]:
spain20_ad['Admitidas'] = spain20_ad['H'] + spain20_ad['M']
spain20_ad = spain20_ad.drop(columns=['H', 'M'])
spain20_ad.head(2)

,Nacionalidad,Admitidas
0,Afganistán,9.0
1,Albania,0.0


In [3382]:
spain20_ad = spain20_ad.fillna(0)
spain20_ad.isnull().sum()

Nacionalidad     0
Admitidas        0
dtype: int64

In [3383]:
spain20_ad = procesar_nacionalidad(spain20_ad)
spain20_ad.head(3)

,Nacionalidad,Admitidas
0,afganistan,9.0
1,albania,0.0
2,angola,21.0


In [3384]:
comprobar_pais_año(spain20, spain20_ad)

8                           azerbaiyan
12                bosnia y herzegovina
39                       guinea-bissau
47                              italia
52                          kirguistan
68         no reconocido (desconocido)
69              no reconocido (kosovo)
70              no reconocido (sahara)
71               no reconocido (tibet)
73                      palestina eonu
77                            portugal
78                               qatar
80     republica democratica del congo
88                           sudafrica
93                   trinidad y tobago
103                           zimbabwe
Name: Nacionalidad , dtype: object


In [3385]:
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('no reconocido (tibet)', 'tibet')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('palestina eonu', 'palestina')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('no reconocido (desconocido)', 'desconocido')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain20_ad['Nacionalidad '] = spain20_ad['Nacionalidad '].replace('republica democratica del congo', 'r.d. congo')

In [3386]:
comprobar_pais_año(spain20, spain20_ad)

47                italia
69                kosovo
70                sahara
71                 tibet
77              portugal
78                 qatar
88             sudafrica
93     trinidad y tobago
103             zimbabwe
Name: Nacionalidad , dtype: object


In [3387]:
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'italia']
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'kosovo']
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'sahara']
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'tibet']
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'portugal']
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'qatar']
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'sudafrica']
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'trinidad y tobago']
spain20_ad = spain20_ad.loc[spain20_ad['Nacionalidad '] != 'zimbabwe']

In [3388]:
comprobar_pais_año(spain20_ad, spain20)

2                   alemania
10                   bahrein
12                    belice
13                     benin
19                   burundi
26                   comoras
28                     corea
36    emiratos arabes unidos
42                     gabon
57                     japon
62                    kuwait
66                 macedonia
88                    ruanda
91                    serbia
98                 tailandia
99                  tanzania
Name: Nacionalidad , dtype: object


In [3389]:
alemania = {'Nacionalidad ': 'alemania', 'Admitidas': -1}
bahrein = {'Nacionalidad ': 'bahrein', 'Admitidas': -1}
belice = {'Nacionalidad ': 'belice', 'Admitidas': -1}
benin = {'Nacionalidad ': 'benin', 'Admitidas': -1}
burundi = {'Nacionalidad ': 'burundi', 'Admitidas': -1}
comoras = {'Nacionalidad ': 'comoras', 'Admitidas': -1}
corea = {'Nacionalidad ': 'corea', 'Admitidas': -1}
emir = {'Nacionalidad ': 'emiratos arabes unidos', 'Admitidas': -1}
gabon = {'Nacionalidad ': 'gabon', 'Admitidas': -1}
japon = {'Nacionalidad ': 'japon', 'Admitidas': -1}
kuwait = {'Nacionalidad ': 'kuwait', 'Admitidas': -1}
macedonia = {'Nacionalidad ': 'macedonia', 'Admitidas': -1}
ruanda = {'Nacionalidad ': 'ruanda', 'Admitidas': -1}
serbia = {'Nacionalidad ': 'serbia', 'Admitidas': -1}
tailandia = {'Nacionalidad ': 'tailandia', 'Admitidas': -1}
tanzania = {'Nacionalidad ': 'tanzania', 'Admitidas': -1}

spain20_ad = pd.concat([spain20_ad, pd.DataFrame([alemania])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([bahrein])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([belice])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([benin])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([burundi])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([comoras])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([corea])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([emir])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([gabon])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([japon])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([kuwait])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([macedonia])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([ruanda])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([serbia])], ignore_index=True)
spain20_ad = pd.concat([spain20_ad, pd.DataFrame([tailandia])], ignore_index=True)

In [3390]:
comprobar_pais_año(spain20_ad, spain20)

99    tanzania
Name: Nacionalidad , dtype: object


In [3391]:
spain20_df = pd.merge(spain20, spain20_ad, on='Nacionalidad ')
spain20_df['Año'] = 2020
spain20_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,27.0,7.0,34.0,9.0,2020
1,albania,37.0,13.0,50.0,0.0,2020


In [3392]:
spain20_df = spain20_df.iloc[:-1]

#### Concat dfs

In [3393]:
frames = [spain12_df, spain13_df, spain14_df, spain15_df, spain16_df, spain17_df, spain18_df, spain19_df, spain20_df]
spain_df = pd.concat(frames)
spain_df.shape

(908, 6)

In [3394]:
countries = sorted(list(set(spain_df['Nacionalidad '].values.tolist())))
countries[0:10]

['afganistan',
 'albania',
 'alemania',
 'angola',
 'apatrida',
 'arabia saudi',
 'argelia',
 'argentina',
 'armenia',
 'azerbaidyan']

### 2021 - España

In [3395]:
spain21 = pd.read_excel(f'../data/raw/Asilo2021.xlsx', sheet_name=1, skiprows=1, usecols='A:D')
spain21 = spain21.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain21.head(3)

,Nacionalidad,Hombres,Mujeres,Total
0,Afganistán,905,811,1716
1,Albania,49,18,67
2,Alemania,1,0,1


In [3396]:
spain21 = procesar_nacionalidad(spain21)
spain21.head(3)

,Nacionalidad,Hombres,Mujeres,Total
0,afganistan,905,811,1716
1,albania,49,18,67
2,alemania,1,0,1


In [3397]:
comprobar_pais(spain21, countries)

,Nacionalidad,Hombres,Mujeres,Total
8,austria,0,1,1
9,azerbaiyan,10,7,17
11,belarus,75,58,133
15,bosnia y herzegovina,2,0,2
21,camboya,0,1,1
27,comores,1,1,2
34,dominica,1,0,1
51,guinea-bissau,118,8,126
64,kirguistan,6,3,9
65,letonia,1,1,2


In [3398]:
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('comores', 'comoras')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('dominica', 'republica dominicana')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('no reconocido (desconocido)', 'desconocido')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain21['Nacionalidad '] = spain21['Nacionalidad '].replace('palestina eonu', 'palestina')

In [3399]:
comprobar_pais(spain21, countries)

,Nacionalidad,Hombres,Mujeres,Total
8,austria,0,1,1
11,belarus,75,58,133
21,camboya,0,1,1
65,letonia,1,1,2
84,paises bajos,2,0,2
95,republica eslovaca,0,1,1
122,total,41930,23552,65482


In [3400]:
spain21_ad = pd.read_excel(f'../data/raw/Asilo2021.xlsx', sheet_name=7, skiprows=2, usecols='A:G')
spain21_ad = spain21_ad.rename(columns={ 'País de origen': 'Nacionalidad ' })
spain21_ad.tail(5)

,Unnamed: 0,H,M,H.1,M.1,H.2,M.2
117,Venezuela,6791,8877,0,1,4,14
118,Vietnam,8,1,0,0,0,0
119,Yemen,86,47,0,0,0,0
120,Zimbabwe,1,0,0,0,0,0
121,Total,38858,26102,23,10,826,175


In [3401]:
spain21_ad = spain21_ad.rename(columns={'Unnamed: 0': 'Nacionalidad ' })
spain21_ad = spain21_ad.drop(columns=['H.1', 'M.1',	'H.2', 'M.2'])
spain21_ad.head(2)

,Nacionalidad,H,M
0,Afganistán,804,694
1,Albania,42,18


In [3402]:
spain21_ad['Admitidas'] = spain21_ad['H'] + spain21_ad['M']
spain21_ad = spain21_ad.drop(columns=['H', 'M'])
spain21_ad.head(2)

,Nacionalidad,Admitidas
0,Afganistán,1498
1,Albania,60


In [3403]:
spain21_ad = procesar_nacionalidad(spain21_ad)
spain21_ad.head(3)

,Nacionalidad,Admitidas
0,afganistan,1498
1,albania,60
2,alemania,1


In [3404]:
comprobar_pais_año(spain21, spain21_ad)

8                      azerbaiyan
13           bosnia y herzegovina
19                         canada
24                        comores
31                       dominica
48                  guinea-bissau
60                     kirguistan
61                         kuwait
78    no reconocido (desconocido)
79         no reconocido (kosovo)
80         no reconocido (sahara)
81                           oman
84                 palestina eonu
94                         ruanda
Name: Nacionalidad , dtype: object


In [3405]:
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('azerbaiyan', 'azerbaidyan')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('bosnia y herzegovina', 'bosnia herzegovina')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('comores', 'comoras')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('dominica', 'republica dominicana')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('guinea-bissau', 'guinea bissau')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('kirguistan', 'kirguizia')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('no reconocido (desconocido)', 'desconocido')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('no reconocido (kosovo)', 'kosovo')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('no reconocido (sahara)', 'sahara')
spain21_ad['Nacionalidad '] = spain21_ad['Nacionalidad '].replace('palestina eonu', 'palestina')

In [3406]:
comprobar_pais_año(spain21, spain21_ad)

19    canada
61    kuwait
81      oman
94    ruanda
Name: Nacionalidad , dtype: object


In [3407]:
spain21_ad = spain21_ad.loc[spain21_ad['Nacionalidad '] != 'canada']
spain21_ad = spain21_ad.loc[spain21_ad['Nacionalidad '] != 'kuwait']
spain21_ad = spain21_ad.loc[spain21_ad['Nacionalidad '] != 'oman']
spain21_ad = spain21_ad.loc[spain21_ad['Nacionalidad '] != 'ruanda']
spain21_ad.shape

(118, 2)

In [3408]:
comprobar_pais_año(spain21_ad, spain21)

8       austria
12      belgica
17     bulgaria
21      camboya
55    indonesia
Name: Nacionalidad , dtype: object


In [3409]:
austria = {'Nacionalidad ': 'austria', 'Admitidas': -1}
belgica = {'Nacionalidad ': 'belgica', 'Admitidas': -1}
bulgaria = {'Nacionalidad ': 'bulgaria', 'Admitidas': -1}
camboya = {'Nacionalidad ': 'camboya', 'Admitidas': -1}
indonesia = {'Nacionalidad ': 'indonesia', 'Admitidas': -1}

spain21_ad = pd.concat([spain21_ad, pd.DataFrame([austria])], ignore_index=True)
spain21_ad = pd.concat([spain21_ad, pd.DataFrame([belgica])], ignore_index=True)
spain21_ad = pd.concat([spain21_ad, pd.DataFrame([bulgaria])], ignore_index=True)
spain21_ad = pd.concat([spain21_ad, pd.DataFrame([camboya])], ignore_index=True)
spain21_ad = pd.concat([spain21_ad, pd.DataFrame([indonesia])], ignore_index=True)

In [3410]:
spain21_ad.shape

(123, 2)

In [3411]:
spain21_df = pd.merge(spain21, spain21_ad, on='Nacionalidad ')
spain21_df['Año'] = 2021
spain21_df.head(2)

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,905,811,1716,1498,2021
1,albania,49,18,67,60,2021


In [3412]:
spain21_df = spain21_df.iloc[:-1]

## Guardar preproceso

In [3413]:
frames = [spain12_df, spain13_df, spain14_df, spain15_df, spain16_df, spain17_df, spain18_df, spain19_df, spain20_df, spain21_df]
spain_df = pd.concat(frames)
spain_df.shape

(1034, 6)

In [3418]:
spain_df[["Hombres", "Mujeres", "Total", "Admitidas"]] = spain_df[["Hombres", "Mujeres", "Total", "Admitidas"]].astype(int)
spain_df[spain_df['Año'] == 2020]

,Nacionalidad,Hombres,Mujeres,Total,Admitidas,Año
0,afganistan,27,7,34,9,2020
1,albania,37,13,50,0,2020
2,alemania,1,0,1,-1,2020
3,angola,13,11,24,21,2020
4,apatrida,5,2,7,0,2020
...,...,...,...,...,...,...
107,uruguay,83,74,157,0,2020
108,uzbekistan,4,5,9,0,2020
109,venezuela,12906,15469,28375,5,2020
110,vietnam,14,9,23,0,2020


In [3419]:
spain_df.to_csv("../data/process/AsiloEspaña.csv", sep=',')